In [3]:
import requests
import pandas as pd
import numpy as np
from keys import *

#Nombre de la Ciudad 
city = "Naucalpan"

#Abreviatura de del PAIS
country = "MX"

#Se pide el clima a la página web/
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [4]:
#Guardam los datos del clima en formato JSON
data = response.json()
#print(data)

# Extraemos la lista del pronóstico del clima
forecast_list = data.get('list', [])

# Preparamos listas para las columnas del DataFrame
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

# extraemos los datos para cada columna, tamperatura, humedad, etc 

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Creamos DataFrame
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"Naucalpan,MX"
})

print(df.head())

      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-10-30 00:00:00        20.08        26         Clouds  5.32       NaN   
1  2025-10-30 03:00:00        17.65        37         Clouds  3.63       NaN   
2  2025-10-30 06:00:00        14.43        54         Clouds  2.05       NaN   
3  2025-10-30 09:00:00         7.56        70         Clouds  1.45       NaN   
4  2025-10-30 12:00:00         5.97        75         Clouds  1.33       NaN   

   snow municipality_iso_country  
0   NaN             Naucalpan,MX  
1   NaN             Naucalpan,MX  
2   NaN             Naucalpan,MX  
3   NaN             Naucalpan,MX  
4   NaN             Naucalpan,MX  


In [15]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "Danis0210"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


In [17]:
# send the weather data to the database
df.to_sql('weather_data', if_exists = 'append', con = con, index=False)

40